互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息 机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。

fraudulent.csv在data文件夹中。

fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：
contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
traffic：该网站的流量大小。1表示大，0表示小；
google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；

y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。

1.原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。

In [1]:
import pandas as pd

# 读取数据集
data = pd.read_csv('D:\\Git-sub\\ItDSaE\\hw11_fraudulent.csv')

# 查看数据的基本信息
print(data.info())

# 查看前几行数据
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10086 entries, 0 to 10085
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   contain_IP             9996 non-null   float64
 1   is_long                9997 non-null   float64
 2   is_tinyurl             9998 non-null   float64
 3   contain_at             10004 non-null  float64
 4   contain_double_slash   9970 non-null   float64
 5   contain_dash           9992 non-null   float64
 6   contain_subdomain      9989 non-null   float64
 7   is_SSL                 9990 non-null   float64
 8   with_long_history      7291 non-null   float64
 9   contain_icon           8728 non-null   float64
 10  contain_ext_domain     8559 non-null   float64
 11  contain_email_to       8007 non-null   float64
 12  allow_right_click      6679 non-null   float64
 13  contain_pop_up_window  9807 non-null   float64
 14  contain_Iframe         9427 non-null   float64
 15  ha

In [2]:
from sklearn.impute import SimpleImputer
# 'with_long_history'和'allow_right_click'两列缺失值较多，直接删除
data = data.drop(['with_long_history', 'allow_right_click'], axis=1)

# 使用众数填充缺失值
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# 查看是否还有缺失值
print(data_imputed.isnull().sum())

contain_IP               0
is_long                  0
is_tinyurl               0
contain_at               0
contain_double_slash     0
contain_dash             0
contain_subdomain        0
is_SSL                   0
contain_icon             0
contain_ext_domain       0
contain_email_to         0
contain_pop_up_window    0
contain_Iframe           0
has_DNSRecord            0
traffic                  0
google_rank              0
y                        0
dtype: int64


2.将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

In [3]:
from sklearn.model_selection import train_test_split
# 提取特征和标签
X = data_imputed.drop(columns=['y'])  # 特征
y = data_imputed['y']  # 标签

# 划分训练集和测试集，设置随机种子为1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# 查看训练集和测试集的大小
print(f"训练集大小: {X_train.shape}")
print(f"测试集大小: {X_test.shape}")


训练集大小: (8068, 16)
测试集大小: (2018, 16)


3.现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。
分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

In [4]:
# k-近邻
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
# 标准化特征
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 创建并训练 KNN 模型
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# 预测测试集
y_pred_knn = knn.predict(X_test_scaled)

# 计算 F1 值
f1_knn = f1_score(y_test, y_pred_knn)
print(f"KNN F1 Score: {f1_knn:.4f}")

KNN F1 Score: 0.8396


In [5]:
# 决策树
from sklearn.tree import DecisionTreeClassifier

# 创建并训练决策树模型
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train, y_train)

# 预测测试集
y_pred_dt = dt.predict(X_test)

# 计算 F1 值
f1_dt = f1_score(y_test, y_pred_dt)
print(f"Decision Tree F1 Score: {f1_dt:.4f}")

Decision Tree F1 Score: 0.8671


In [6]:
# 逻辑回归
from sklearn.linear_model import LogisticRegression
# 创建并训练逻辑回归模型
lr = LogisticRegression(random_state=1)
lr.fit(X_train_scaled, y_train)

# 预测测试集
y_pred_lr = lr.predict(X_test_scaled)

# 计算 F1 值
f1_lr = f1_score(y_test, y_pred_lr)
print(f"Logistic Regression F1 Score: {f1_lr:.4f}")

Logistic Regression F1 Score: 0.8500


In [7]:
# 支持向量机
from sklearn.svm import SVC
# 创建并训练支持向量机模型
svm = SVC(random_state=1)
svm.fit(X_train_scaled, y_train)

# 预测测试集
y_pred_svm = svm.predict(X_test_scaled)

# 计算 F1 值
f1_svm = f1_score(y_test, y_pred_svm)
print(f"SVM F1 Score: {f1_svm:.4f}")

SVM F1 Score: 0.8610


4.评估指标为F1值
可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

In [8]:
# 打印各个模型的 F1 分数
f1_scores = {
    "KNN": f1_knn,
    "Decision Tree": f1_dt,
    "Logistic Regression": f1_lr,
    "SVM": f1_svm
}

# 输出 F1 分数
print("\nF1 Scores for different models:")
for model, score in f1_scores.items():
    print(f"{model}: {score:.4f}")

# 输出最佳模型
best_model = max(f1_scores, key=f1_scores.get)
print(f"\nBest Model: {best_model} with F1 Score: {f1_scores[best_model]:.4f}")



F1 Scores for different models:
KNN: 0.8396
Decision Tree: 0.8671
Logistic Regression: 0.8500
SVM: 0.8610

Best Model: Decision Tree with F1 Score: 0.8671
